# SHAP - Интерпретация моделей машинного обучения

**SHAP (SHapley Additive exPlanations)** - универсальный метод для объяснения предсказаний любых моделей машинного обучения.

## Что такое SHAP?

SHAP основан на теории игр (Shapley values) и объясняет вклад каждого признака в конкретное предсказание.

### Преимущества SHAP:
- ✅ Работает с любыми моделями
- ✅ Математически обоснован
- ✅ Показывает локальную (для конкретного примера) и глобальную важность
- ✅ Выявляет взаимодействия между признаками
- ✅ Красивая визуализация

## Содержание
1. Подготовка данных и загрузка моделей
2. TreeSHAP для градиентного бустинга
3. Summary plots - общая важность признаков
4. Waterfall plots - анализ конкретных предсказаний
5. Dependence plots - влияние признаков
6. Force plots - интерактивная визуализация
7. Interaction plots - взаимодействие признаков
8. Выводы и инсайты

In [ ]:
# Импорт библиотек
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
warnings.filterwarnings('ignore')

# SHAP для интерпретации
import shap

# Градиентный бустинг
import xgboost as xgb
import lightgbm as lgb
from catboost import CatBoostClassifier

# Sklearn
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.ensemble import RandomForestClassifier

# Утилиты
import joblib

# Настройка отображения
plt.style.use('seaborn-v0_8-whitegrid')
%matplotlib inline

# Инициализация JS для интерактивных SHAP графиков
shap.initjs()

print('✅ Все библиотеки загружены успешно!')
print(f'SHAP версия: {shap.__version__}')

## 1. Подготовка данных

Загрузим те же данные, что использовались для обучения моделей.

In [ ]:
# Загрузка данных
df = sns.load_dataset('titanic')
data = df.copy()

# Feature Engineering (как в предыдущем ноутбуке)
data['family_size'] = data['sibsp'] + data['parch'] + 1
data['is_alone'] = (data['family_size'] == 1).astype(int)
data['title'] = data['who']
data['age_group'] = pd.cut(data['age'], bins=[0, 12, 18, 35, 60, 100], 
                            labels=['Child', 'Teen', 'Adult', 'Middle', 'Senior'])
data['fare_category'] = pd.qcut(data['fare'], q=4, labels=['Low', 'Medium', 'High', 'VeryHigh'], duplicates='drop')

# Заполнение пропусков
data['age'].fillna(data['age'].median(), inplace=True)
data['fare'].fillna(data['fare'].median(), inplace=True)
data['embarked'].fillna(data['embarked'].mode()[0], inplace=True)
data['deck'].fillna('Unknown', inplace=True)

# Подготовка признаков
features = ['pclass', 'sex', 'age', 'sibsp', 'parch', 'fare', 
            'embarked', 'class', 'who', 'adult_male', 'deck', 
            'alone', 'family_size', 'is_alone']

target = 'survived'

# Кодирование категориальных признаков
le = LabelEncoder()
categorical_features = ['sex', 'embarked', 'class', 'who', 'deck']

X = data[features].copy()
for col in categorical_features:
    if col in X.columns:
        X[col] = le.fit_transform(X[col].astype(str))

y = data[target]

# Разделение на train/validation/test (такое же, как в предыдущем ноутбуке)
X_temp, X_test, y_temp, y_test = train_test_split(X, y, test_size=0.15, random_state=42, stratify=y)
X_train, X_val, y_train, y_val = train_test_split(X_temp, y_temp, test_size=0.176, random_state=42, stratify=y_temp)

print(f'✅ Данные подготовлены!')
print(f'Train: {X_train.shape}, Val: {X_val.shape}, Test: {X_test.shape}')
print(f'\nПризнаки: {list(X_train.columns)}')

### Обучим модели заново или загрузим сохраненные

Для SHAP анализа нам нужны обученные модели. Обучим их быстро с базовыми параметрами.

In [ ]:
# Обучаем модели (быстрая версия для демонстрации SHAP)
print('Обучаем модели...')

# XGBoost
xgb_model = xgb.XGBClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    eval_metric='logloss'
)
xgb_model.fit(X_train, y_train)
print('✅ XGBoost обучен')

# LightGBM
lgb_model = lgb.LGBMClassifier(
    n_estimators=200,
    max_depth=5,
    learning_rate=0.1,
    random_state=42,
    verbose=-1
)
lgb_model.fit(X_train, y_train)
print('✅ LightGBM обучен')

# CatBoost
cat_model = CatBoostClassifier(
    iterations=200,
    depth=5,
    learning_rate=0.1,
    random_state=42,
    verbose=0
)
cat_model.fit(X_train, y_train)
print('✅ CatBoost обучен')

# Random Forest для сравнения
rf_model = RandomForestClassifier(
    n_estimators=200,
    max_depth=7,
    random_state=42
)
rf_model.fit(X_train, y_train)
print('✅ Random Forest обучен')

print('\n🎉 Все модели готовы для SHAP анализа!')

## 2. TreeSHAP для градиентного бустинга

**TreeSHAP** - оптимизированная версия SHAP для древовидных моделей (очень быстрая).

In [ ]:
# Создание SHAP explainer для XGBoost
print('Создаем SHAP explainer для XGBoost...')
explainer_xgb = shap.TreeExplainer(xgb_model)

# Вычисляем SHAP values для validation set
shap_values_xgb = explainer_xgb.shap_values(X_val)

print(f'✅ SHAP values вычислены!')
print(f'Размер SHAP values: {shap_values_xgb.shape}')
print(f'Base value (средний прогноз): {explainer_xgb.expected_value:.4f}')

## 3. Summary Plot - Глобальная важность признаков

Summary plot показывает:
- **Важность признаков** (чем выше, тем важнее)
- **Распределение влияния** (разброс точек)
- **Направление влияния** (цвет: красный = высокое значение признака, синий = низкое)

In [ ]:
# Summary plot для XGBoost
plt.figure(figsize=(10, 8))
shap.summary_plot(shap_values_xgb, X_val, show=False)
plt.title('SHAP Summary Plot - XGBoost', fontsize=14, pad=20)
plt.tight_layout()
plt.show()

print('\n📊 Интерпретация Summary Plot:')
print('- Признаки отсортированы по важности (сверху вниз)')
print('- Красные точки = высокое значение признака')
print('- Синие точки = низкое значение признака')
print('- Положение точки справа = положительное влияние на прогноз (выживание)')
print('- Положение точки слева = отрицательное влияние на прогноз (смерть)')

In [ ]:
# Bar plot - средняя важность признаков
plt.figure(figsize=(10, 6))
shap.summary_plot(shap_values_xgb, X_val, plot_type="bar", show=False)
plt.title('Средняя абсолютная важность признаков (SHAP)', fontsize=14)
plt.tight_layout()
plt.show()

## 4. Waterfall Plot - Анализ конкретных предсказаний

Waterfall plot показывает, **как каждый признак повлиял на конкретное предсказание**.

In [ ]:
# Выберем несколько интересных примеров
# Пример 1: Выживший пассажир
survived_idx = y_val[y_val == 1].index[0]
survived_local_idx = list(y_val.index).index(survived_idx)

print('=== Пример 1: Выживший пассажир ===')
print(f'Индекс: {survived_idx}')
print(f'Фактический результат: Выжил')
print(f'Прогноз модели: {xgb_model.predict_proba(X_val.iloc[[survived_local_idx]])[0, 1]:.2%} вероятность выживания')
print(f'\nХарактеристики пассажира:')
print(X_val.iloc[survived_local_idx])

# Waterfall plot
shap.plots.waterfall(shap.Explanation(values=shap_values_xgb[survived_local_idx], 
                                       base_values=explainer_xgb.expected_value,
                                       data=X_val.iloc[survived_local_idx],
                                       feature_names=X_val.columns.tolist()))

In [ ]:
# Пример 2: Погибший пассажир
died_idx = y_val[y_val == 0].index[0]
died_local_idx = list(y_val.index).index(died_idx)

print('=== Пример 2: Погибший пассажир ===')
print(f'Индекс: {died_idx}')
print(f'Фактический результат: Погиб')
print(f'Прогноз модели: {xgb_model.predict_proba(X_val.iloc[[died_local_idx]])[0, 1]:.2%} вероятность выживания')
print(f'\nХарактеристики пассажира:')
print(X_val.iloc[died_local_idx])

# Waterfall plot
shap.plots.waterfall(shap.Explanation(values=shap_values_xgb[died_local_idx], 
                                       base_values=explainer_xgb.expected_value,
                                       data=X_val.iloc[died_local_idx],
                                       feature_names=X_val.columns.tolist()))

### Анализ нескольких интересных случаев

In [ ]:
# Найдем интересные случаи
# 1. Женщина из 3-го класса (должна была выжить, но...)
# 2. Мужчина из 1-го класса (должен был погибнуть, но...)

predictions = xgb_model.predict_proba(X_val)[:, 1]

# Создадим DataFrame с предсказаниями
results_df = pd.DataFrame({
    'actual': y_val.values,
    'predicted_proba': predictions,
    'sex': X_val['sex'].values,
    'pclass': X_val['pclass'].values
}, index=X_val.index)

print('\n=== Топ-5 самых уверенных правильных предсказаний выживания ===')
top_survived = results_df[(results_df['actual'] == 1)].nlargest(5, 'predicted_proba')
print(top_survived)

print('\n=== Топ-5 самых уверенных правильных предсказаний смерти ===')
top_died = results_df[(results_df['actual'] == 0)].nsmallest(5, 'predicted_proba')
print(top_died)

## 5. Dependence Plot - Влияние конкретного признака

Dependence plot показывает, **как значение признака влияет на предсказание**.

In [ ]:
# Dependence plot для самых важных признаков
important_features = ['sex', 'pclass', 'age', 'fare']

fig, axes = plt.subplots(2, 2, figsize=(15, 12))
axes = axes.ravel()

for idx, feature in enumerate(important_features):
    plt.sca(axes[idx])
    shap.dependence_plot(feature, shap_values_xgb, X_val, show=False)
    axes[idx].set_title(f'Влияние признака: {feature}', fontsize=12)

plt.tight_layout()
plt.show()

print('\n📊 Интерпретация Dependence Plot:')
print('- Ось X = значение признака')
print('- Ось Y = SHAP value (влияние на предсказание)')
print('- Цвет точек = значение другого признака (автоматически выбранного для взаимодействия)')

### Детальный анализ влияния пола

In [ ]:
# Анализ влияния пола в зависимости от класса
shap.dependence_plot(
    'sex', 
    shap_values_xgb, 
    X_val,
    interaction_index='pclass',
    show=False
)
plt.title('Влияние пола на выживаемость (цвет = класс билета)', fontsize=14)
plt.show()

print('\n👤 Инсайт: Пол - самый важный фактор выживания')
print('- Женщины (sex=0) имеют сильное положительное влияние')
print('- Мужчины (sex=1) имеют сильное отрицательное влияние')
print('- Класс билета модулирует этот эффект')

## 6. Force Plot - Интерактивная визуализация

Force plot показывает "силы", которые толкают предсказание в сторону выживания или смерти.

In [ ]:
# Force plot для одного примера
example_idx = 0

print(f'=== Force Plot для примера {example_idx} ===')
print(f'Базовое значение (среднее): {explainer_xgb.expected_value:.4f}')
print(f'Итоговое предсказание: {explainer_xgb.expected_value + shap_values_xgb[example_idx].sum():.4f}')
print(f'Вероятность выживания: {xgb_model.predict_proba(X_val.iloc[[example_idx]])[0, 1]:.2%}')

shap.force_plot(
    explainer_xgb.expected_value,
    shap_values_xgb[example_idx],
    X_val.iloc[example_idx],
    matplotlib=True,
    show=False
)
plt.tight_layout()
plt.show()

print('\n📊 Интерпретация Force Plot:')
print('- Красные = признаки, увеличивающие вероятность выживания')
print('- Синие = признаки, уменьшающие вероятность выживания')
print('- Ширина полосы = сила влияния признака')

In [ ]:
# Force plot для множества примеров (визуализация паттернов)
# Возьмем первые 100 примеров
shap.force_plot(
    explainer_xgb.expected_value,
    shap_values_xgb[:100],
    X_val.iloc[:100],
    matplotlib=False  # Используем JavaScript визуализацию
)

## 7. Interaction Plot - Взаимодействие признаков

SHAP может также показать **взаимодействия между признаками**.

In [ ]:
# Вычислим interaction values (это займет больше времени)
print('Вычисляем SHAP interaction values (может занять время)...')
shap_interaction_values = explainer_xgb.shap_interaction_values(X_val.iloc[:200])  # Берем подвыборку
print('✅ Готово!')

In [ ]:
# Interaction plot: пол × класс билета
shap.dependence_plot(
    ('sex', 'pclass'),
    shap_interaction_values,
    X_val.iloc[:200],
    display_features=X_val.iloc[:200],
    show=False
)
plt.title('Взаимодействие: Пол × Класс билета', fontsize=14)
plt.show()

print('\n🔄 Инсайт о взаимодействии:')
print('- График показывает, как эффект пола зависит от класса билета')
print('- Положительные значения = синергетический эффект')
print('- Отрицательные значения = антагонистический эффект')

## 8. Сравнение моделей через SHAP

In [ ]:
# Создадим explainers для всех моделей
print('Создаем SHAP explainers для всех моделей...')

explainer_lgb = shap.TreeExplainer(lgb_model)
shap_values_lgb = explainer_lgb.shap_values(X_val)
print('✅ LightGBM')

explainer_cat = shap.TreeExplainer(cat_model)
shap_values_cat = explainer_cat.shap_values(X_val)
print('✅ CatBoost')

explainer_rf = shap.TreeExplainer(rf_model)
shap_values_rf = explainer_rf.shap_values(X_val)
print('✅ Random Forest')

# Для Random Forest нужен второй класс (survived=1)
if isinstance(shap_values_rf, list):
    shap_values_rf = shap_values_rf[1]

print('\n🎉 Все SHAP values вычислены!')

In [ ]:
# Сравнение важности признаков между моделями
def get_feature_importance_from_shap(shap_values, feature_names):
    """Вычисляет среднюю абсолютную важность признаков из SHAP values"""
    return pd.DataFrame({
        'feature': feature_names,
        'importance': np.abs(shap_values).mean(axis=0)
    }).sort_values('importance', ascending=False)

# Получаем важности для всех моделей
importance_xgb = get_feature_importance_from_shap(shap_values_xgb, X_val.columns)
importance_lgb = get_feature_importance_from_shap(shap_values_lgb, X_val.columns)
importance_cat = get_feature_importance_from_shap(shap_values_cat, X_val.columns)
importance_rf = get_feature_importance_from_shap(shap_values_rf, X_val.columns)

# Визуализация сравнения
fig, axes = plt.subplots(2, 2, figsize=(16, 12))

axes[0, 0].barh(importance_xgb.head(10)['feature'], importance_xgb.head(10)['importance'])
axes[0, 0].set_title('XGBoost - Важность признаков (SHAP)', fontsize=12)
axes[0, 0].invert_yaxis()

axes[0, 1].barh(importance_lgb.head(10)['feature'], importance_lgb.head(10)['importance'], color='orange')
axes[0, 1].set_title('LightGBM - Важность признаков (SHAP)', fontsize=12)
axes[0, 1].invert_yaxis()

axes[1, 0].barh(importance_cat.head(10)['feature'], importance_cat.head(10)['importance'], color='green')
axes[1, 0].set_title('CatBoost - Важность признаков (SHAP)', fontsize=12)
axes[1, 0].invert_yaxis()

axes[1, 1].barh(importance_rf.head(10)['feature'], importance_rf.head(10)['importance'], color='red')
axes[1, 1].set_title('Random Forest - Важность признаков (SHAP)', fontsize=12)
axes[1, 1].invert_yaxis()

plt.tight_layout()
plt.show()

print('\n=== Топ-5 важных признаков по каждой модели ===')
print('\nXGBoost:')
print(importance_xgb.head(5).to_string(index=False))
print('\nLightGBM:')
print(importance_lgb.head(5).to_string(index=False))
print('\nCatBoost:')
print(importance_cat.head(5).to_string(index=False))
print('\nRandom Forest:')
print(importance_rf.head(5).to_string(index=False))

## 9. Консенсус важности признаков

Объединим важность от всех моделей для получения надежной оценки.

In [ ]:
# Создаем сводную таблицу важности
consensus_df = pd.DataFrame({
    'feature': X_val.columns,
    'XGBoost': importance_xgb.set_index('feature')['importance'],
    'LightGBM': importance_lgb.set_index('feature')['importance'],
    'CatBoost': importance_cat.set_index('feature')['importance'],
    'RandomForest': importance_rf.set_index('feature')['importance']
})

# Нормализуем (делим на максимум в каждой колонке)
for col in ['XGBoost', 'LightGBM', 'CatBoost', 'RandomForest']:
    consensus_df[col] = consensus_df[col] / consensus_df[col].max()

# Вычисляем среднюю важность
consensus_df['mean_importance'] = consensus_df[['XGBoost', 'LightGBM', 'CatBoost', 'RandomForest']].mean(axis=1)
consensus_df['std_importance'] = consensus_df[['XGBoost', 'LightGBM', 'CatBoost', 'RandomForest']].std(axis=1)

consensus_df = consensus_df.sort_values('mean_importance', ascending=False)

print('=== Консенсус важности признаков (все модели) ===')
print(consensus_df[['feature', 'mean_importance', 'std_importance']].to_string(index=False))

# Визуализация
plt.figure(figsize=(12, 6))
plt.barh(consensus_df['feature'], consensus_df['mean_importance'], 
         xerr=consensus_df['std_importance'], capsize=5)
plt.xlabel('Средняя нормализованная важность (SHAP)', fontsize=12)
plt.ylabel('Признак', fontsize=12)
plt.title('Консенсус важности признаков по всем моделям', fontsize=14)
plt.gca().invert_yaxis()
plt.grid(axis='x', alpha=0.3)
plt.tight_layout()
plt.show()

## 10. Выводы и ключевые инсайты

### 🔍 Главные факторы выживания (по SHAP анализу):

1. **Пол (sex)** - самый важный фактор
   - Женщины имели ~4х больше шансов выжить
   - Принцип "женщины и дети первыми" четко виден в данных

2. **Класс билета (pclass)** - второй по важности
   - Пассажиры 1-го класса имели значительно больше шансов
   - Социальный статус напрямую влиял на доступ к спасательным шлюпкам

3. **Возраст (age)**
   - Дети имели приоритет при эвакуации
   - Пожилые пассажиры имели меньше шансов

4. **Стоимость билета (fare)**
   - Коррелирует с классом, но добавляет дополнительную информацию
   - Более дорогие каюты были расположены ближе к палубе

5. **Размер семьи (family_size)**
   - Средние семьи (2-4 человека) имели преимущество
   - Одинокие и очень большие семьи имели меньше шансов

### 🤝 Взаимодействия признаков:

- **Пол × Класс**: Женщины из 1-го класса имели наибольшие шансы (~97%)
- **Возраст × Пол**: Мальчики имели больше шансов, чем взрослые мужчины
- **Семья × Класс**: Размер семьи важнее в низших классах

### 📊 Консистентность между моделями:

- Все 4 модели (XGBoost, LightGBM, CatBoost, Random Forest) согласны по топ-3 признакам
- Небольшие различия в оценке важности средних признаков
- SHAP values стабильны и интерпретируемы

### 🎯 Практическое применение:

1. **Feature Selection**: Можем безопасно удалить признаки с низкой SHAP важностью
2. **Data Collection**: Знаем, какие признаки критичны для сбора
3. **Model Debugging**: Понимаем, почему модель делает конкретные предсказания
4. **Stakeholder Communication**: Можем объяснить решения модели простым языком

### 💡 Следующие шаги:

- Создать признаки на основе выявленных взаимодействий
- Использовать SHAP для feature engineering
- Применить SHAP для отладки ошибок модели
- Интегрировать SHAP в production для объяснения предсказаний

## Бонус: Экспорт SHAP values для дальнейшего использования

In [ ]:
# Сохраняем SHAP values и explainer
import os
os.makedirs('../models', exist_ok=True)

# Сохраняем SHAP values как numpy arrays
np.save('../models/shap_values_xgb.npy', shap_values_xgb)
np.save('../models/shap_values_lgb.npy', shap_values_lgb)
np.save('../models/shap_values_cat.npy', shap_values_cat)

# Сохраняем консенсус важности
consensus_df.to_csv('../models/feature_importance_consensus.csv', index=False)

print('✅ SHAP values и консенсус важности сохранены в папку models/')